In [26]:
import pygame
import random

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 700, 900
GRID_SIZE = 17
TILE_SIZE = 40
RACK_SIZE = 14
FONT = pygame.font.Font(None, 36)

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)

class Tile:
    def __init__(self, letter, rect):
        self.letter = letter
        self.rect = rect
        self.dragging = False

class ScrabbleGame:
    def __init__(self):
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        pygame.display.set_caption("Bananagrams")

        # Initialize the game board and rack
        self.board = [[' ' for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        self.rack = self.draw_tiles(RACK_SIZE)
        self.selected_tile = None

        # Main game loop
        self.running = True
        while self.running:
            self.handle_events()
            self.draw_board()
            self.draw_rack()
            pygame.display.flip()

        pygame.quit()

    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                self.handle_mouse_down(event.pos)
            elif event.type == pygame.MOUSEMOTION:
                self.handle_mouse_motion(event.pos)
            elif event.type == pygame.MOUSEBUTTONUP:
                self.handle_mouse_up()

    def handle_mouse_down(self, mouse_pos):
        # Check if a tile on the rack is clicked
        for tile in self.rack:
            if tile.rect.collidepoint(mouse_pos):
                tile.dragging = True
                self.selected_tile = tile
                return

        # Check if a tile on the board is clicked
        for row in range(GRID_SIZE):
            for col in range(GRID_SIZE):
                tile_rect = pygame.Rect(col * TILE_SIZE, row * TILE_SIZE, TILE_SIZE, TILE_SIZE)
                if self.board[row][col] != ' ' and tile_rect.collidepoint(mouse_pos):
                    # Place the selected tile back into the rack
                    letter = self.board[row][col]
                    rect = pygame.Rect(len(self.rack) * TILE_SIZE, HEIGHT - TILE_SIZE, TILE_SIZE, TILE_SIZE)
                    self.rack.append(Tile(letter, rect))
                    self.board[row][col] = ' '  # Remove the tile from the board
                    self.selected_tile = self.rack[-1]
                    self.selected_tile.dragging = True
                    return

    def handle_mouse_motion(self, mouse_pos):
        # Move the selected tile with the mouse
        if self.selected_tile and self.selected_tile.dragging:
            self.selected_tile.rect.x, self.selected_tile.rect.y = mouse_pos

    def handle_mouse_up(self):
        # Drop the selected tile onto the board
        if self.selected_tile:
            for row in range(GRID_SIZE):
                for col in range(GRID_SIZE):
                    if self.board[row][col] == ' ' and self.selected_tile.rect.colliderect(pygame.Rect(col * TILE_SIZE, row * TILE_SIZE, TILE_SIZE, TILE_SIZE)):
                        self.board[row][col] = self.selected_tile.letter
                        self.selected_tile.dragging = False
                        self.rack.remove(self.selected_tile)  # Remove the tile from the rack
                        self.selected_tile = None
                        return
            # If not dropped on the board, return it to the rack
            self.selected_tile.dragging = False
            self.selected_tile = None


    def draw_board(self):
        self.screen.fill(WHITE)

        for row in range(GRID_SIZE):
            for col in range(GRID_SIZE):
                pygame.draw.rect(self.screen, BLACK, (col * TILE_SIZE, row * TILE_SIZE, TILE_SIZE, TILE_SIZE), 1)
                text = FONT.render(self.board[row][col], True, BLACK)
                self.screen.blit(text, (col * TILE_SIZE + TILE_SIZE // 3, row * TILE_SIZE + TILE_SIZE // 3))

    def draw_rack(self):
        
        for i, tile in enumerate(self.rack):
            pygame.draw.rect(self.screen, GRAY, tile.rect, 1)
            text = FONT.render(tile.letter, True, BLACK)
            self.screen.blit(text, (tile.rect.x + TILE_SIZE // 3 , tile.rect.y + TILE_SIZE // 6))

    def draw_tiles(self, num_tiles):
        
        # Simulate drawing tiles from a bag (you may replace this with your own logic)
        tile_bag = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', ]
        rack = []
        for i in range(num_tiles):
            letter = random.choice(tile_bag)
            tile_bag.remove(letter)
            rect = pygame.Rect(i * TILE_SIZE, HEIGHT - TILE_SIZE, TILE_SIZE, TILE_SIZE)
            rack.append(Tile(letter, rect))
        return rack
        
    

if __name__ == "__main__":
    ScrabbleGame()


In [15]:
import pygame
import sys

# Constants
WIDTH, HEIGHT = 800, 600
BOARD_SIZE = 15
TILE_SIZE = WIDTH // BOARD_SIZE
RACK_HEIGHT = 100  # Height of the player's tile rack

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)

class Tile(pygame.sprite.Sprite):
    def __init__(self, letter, row, col):
        super().__init__()
        self.image = pygame.Surface((TILE_SIZE, TILE_SIZE))
        self.image.fill(RED)  # Placeholder color for the tile
        self.rect = self.image.get_rect(topleft=(col * TILE_SIZE, row * TILE_SIZE))
        self.letter = letter

#print(deck.deck)

class Board:
    def __init__(self):
        self.board_data = [[None for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]

    def draw(self, screen):
        for row in range(BOARD_SIZE):
            for col in range(BOARD_SIZE):
                color = WHITE if (row + col) % 2 == 0 else BLACK
                pygame.draw.rect(
                    screen,
                    color,
                    (col * TILE_SIZE, row * TILE_SIZE, TILE_SIZE, TILE_SIZE),
                )

    def place_tile(self, row, col, tile):
        if self.is_valid_move(row, col):
            self.board_data[row][col] = tile
            return True
        else:
            return False

    def is_valid_move(self, row, col):
        # Add your custom logic for checking if the move is valid
        return self.board_data[row][col] is None

class TileRack:
    def __init__(self):
        self.tiles = [Tile(chr(65 + i), 0, 0) for i in range(7)]

    def draw(self, screen):
        # Set the position for the tile rack at the bottom center
        rack_x = WIDTH // 2 - (len(self.tiles) * TILE_SIZE) // 2
        rack_y = HEIGHT - RACK_HEIGHT
        for i, tile in enumerate(self.tiles):
            tile.rect.topleft = (rack_x + i * TILE_SIZE, rack_y)
            screen.blit(tile.image, tile.rect)

# Initialize Pygame
pygame.init()

# Initialize the screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Scrabble Board")

# Create a Scrabble board and a player's tile rack
scrabble_board = Board()
player_rack = TileRack()

# Main game loop
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:  # Left mouse button
                mouse_x, mouse_y = event.pos
                row = mouse_y // TILE_SIZE
                col = mouse_x // TILE_SIZE

                if scrabble_board.place_tile(row, col, player_rack.tiles[0]):
                    print(f"Tile placed at row {row}, col {col}")

    # Draw the board
    scrabble_board.draw(screen)

    # Draw the player's tile rack
    player_rack.draw(screen)

    # Update the display
    pygame.display.flip()


SystemExit: 

/home/slaing/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
import pygame
import sys

# Constants
WIDTH, HEIGHT = 800, 600
BOARD_SIZE = 15
TILE_SIZE = WIDTH // BOARD_SIZE
RACK_HEIGHT = 100  # Height of the player's tile rack

# Colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)

class Tile(pygame.sprite.Sprite):
    def __init__(self, letter, row, col):
        super().__init__()
        self.image = pygame.Surface((TILE_SIZE, TILE_SIZE))
        self.image.fill(RED)  # Placeholder color for the tile
        self.rect = self.image.get_rect(topleft=(col * TILE_SIZE, row * TILE_SIZE))
        self.letter = letter

class Board:
    def __init__(self):
        self.board_data = [[None for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]

    def draw(self, screen):
        for row in range(BOARD_SIZE):
            for col in range(BOARD_SIZE):
                pygame.draw.rect(
                    screen,
                    WHITE,
                    (col * TILE_SIZE, row * TILE_SIZE, TILE_SIZE, TILE_SIZE),
                )

    def place_tile(self, row, col, tile):
        if self.is_valid_move(row, col):
            self.board_data[row][col] = tile
            return True
        else:
            return False

    def is_valid_move(self, row, col):
        # Add your custom logic for checking if the move is valid
        return self.board_data[row][col] is None

class TileRack:
    def __init__(self):
        self.tiles = [Tile(chr(65 + i), 0, 0) for i in range(7)]

    def draw(self, screen):
        # Set the position for the tile rack at the top center
        rack_x = WIDTH // 2 - (len(self.tiles) * TILE_SIZE) // 2
        rack_y = 0  # Adjusted to the top of the screen
        for i, tile in enumerate(self.tiles):
            tile.rect.topleft = (rack_x + i * TILE_SIZE, rack_y)
            screen.blit(tile.image, tile.rect)

# Initialize Pygame
pygame.init()

# Initialize the screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Scrabble Board")

# Create a Scrabble board and a player's tile rack
scrabble_board = Board()
player_rack = TileRack()

# Main game loop
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if event.button == 1:  # Left mouse button
                mouse_x, mouse_y = event.pos
                row = mouse_y // TILE_SIZE
                col = mouse_x // TILE_SIZE

                if scrabble_board.place_tile(row, col, player_rack.tiles[0]):
                    print(f"Tile placed at row {row}, col {col}")

    # Draw the board
    scrabble_board.draw(screen)

    # Draw the player's tile rack
    player_rack.draw(screen)

    # Update the display
    pygame.display.flip()


Tile placed at row 0, col 7


SystemExit: 